In [121]:
import numpy as np
import sys
sys.path.insert(0, '/Library/Application Support/MWorks/Scripting/Python')
from mworks.data import MWKFile
import matplotlib.pyplot as plt
import matplotlib.style as style 
import pandas as pd
import pickle as pkl

In [85]:
style.use('seaborn-notebook')
# plt.gca().spines['top'].set_visible(False)
# plt.gca().spines['right'].set_visible(False)
style.use('seaborn-white')

In [119]:
username = 'apiccato'
# username = 'aidapiccato'
dir_path = '/Users/%s/PyCharmProjects/concentration/concentration-game-mworks' % username
subject_id = 0
subject_fn = '%s/meta/subject_%s.pkl' % (dir_path, subject_id)

In [12]:
## sync times 

ITI = 1
TRIAL_INIT = 2
FLIP_CARD_A = 3
FLIP_CARD_B = 4
FEEDBACK = 5
TRIAL_END = 6

In [252]:
def unpack(fn, subject_fn):
    """
    
    :param fn: MWorks event stream filename 
    :param subject_fn: Subject metadata filename
    :return: 
    """
    def flatten_meta():
        """
        
        :return: Pandas data frame with n_trials rows 
        """
        meta_flat = {'block_index':[], 'grid': [], 'card_a': [], 'trial_index': [], 'n_pairs': [], 'grid_dims': [], 'inv_grid': [],}
        with open(subject_fn, 'rb') as f:
            meta = pkl.load(f)
        for block_index, block in enumerate(meta):
            inv_grid = block['inv_grid']
            grid_dims = block['grid_dims']
            n_pairs = block['n_pairs']
            grid = block['grid']
            for trial_index, card_a in enumerate(block['trials']):
                meta_flat['block_index'].append(block_index)
                meta_flat['trial_index'].append(trial_index)
                meta_flat['n_pairs'].append(n_pairs)
                meta_flat['grid_dims'].append(grid_dims)
                meta_flat['inv_grid'].append(inv_grid)
                meta_flat['grid'].append(grid)
                meta_flat['card_a'].append(card_a)
        return pd.DataFrame(meta_flat)
            
            
    def get_sync_times(f):
        """
        
        :param f: MWKFile object 
        :return: 
            start_sync_t: array of trial start times
            end_sync_t: array of trial end times              
        """
        sync_events = f.get_events(codes=['sync'])
        sync_events = np.asarray([[e.data, e.time] for e in sync_events])        
        end_sync_t = sync_events[np.where(sync_events[:, 0] == TRIAL_END)[0], 1]
        start_sync_t = sync_events[np.where(sync_events[:, 0] == TRIAL_INIT)[0], 1]
        
        start_sync_t = start_sync_t[:len(end_sync_t)]
        
        return start_sync_t, end_sync_t
        
    def get_dist(card_a, card_b, grid_dims):
        """
        Returns spatial distance between two cards for given grid dimension         
        :param grid: 
        :param card_a: 
        :param card_b: 
        :param grid_dims: 
        :return: 
        """
        card_a_loc = np.asarray([np.floor(card_a/grid_dims[1]), card_a%grid_dims[1]])
        card_b_loc = np.asarray([np.floor(card_b/grid_dims[1]), card_b%grid_dims[1]])
        return np.linalg.norm(card_a_loc - card_b_loc)
    
    def get_reaction_times(f, start_sync_t, end_sync_t):
        N = len(start_sync_t)
        rts = []
        for trial in range(N):
            start_t, end_t = np.long(start_sync_t[trial]), np.long(end_sync_t[trial])
            sync = f.get_events(codes = ['sync'], time_range=[start_t, end_t])
            sync = np.asarray([[e.data, e.time] for e in sync])                
            flip_card_a = sync[np.where(sync[:, 0] == FLIP_CARD_A)[0], 1]
            flip_card_b = sync[np.where(sync[:, 0] == FLIP_CARD_B)[0], 1]
            rt = np.long(flip_card_b) - np.long(flip_card_a)
            rts.append(rt/10**6)
        return rts
            
    def get_scalar_data(f, codenames, start_sync_t, end_sync_t):
        N = len(start_sync_t) 
        data_scalar = {'dur': [], 'card_a_img': [], 'card_b_img': [], 'dist': []}
        meta = flatten_meta()
        for c in codenames:
            data_scalar[c] = []
        for c in meta.columns:
            if c != 'card_a':
                data_scalar[c] = []
        for trial in range(N):            
            start_t, end_t = np.long(start_sync_t[trial]), np.long(end_sync_t[trial])
            trial_events = f.get_events(codes = codenames, time_range=[start_t, end_t])
            trial_events = np.asarray([[e.code, e.time, e.data] for e in trial_events])         
            data_scalar['dur'].append((end_t - start_t)/10**6)        
            for ci, c in enumerate(codenames): # currently this is only selecting card_b                
                code = f.reverse_codec[c]
                code_events = trial_events[np.where(trial_events[:, 0] == code)[0], 2]
                data_scalar[c].append(code_events[0])                   
            for ci, c in enumerate(meta.columns):
                if c != 'card_a':
                    data_scalar[c].append(meta.loc[trial][c])
            data_scalar['card_a_img'].append(data_scalar['grid'][trial][data_scalar['card_a'][trial]])
            data_scalar['card_b_img'].append(data_scalar['grid'][trial][data_scalar['card_b'][trial]])
            data_scalar['dist'].append(get_dist(data_scalar['card_a'][trial], data_scalar['card_b'][trial], data_scalar['grid_dims'][trial]))
            # verifying that card_a from meta and event stream line up
            if meta.loc[trial]['card_a'] != data_scalar['card_a'][trial]:
                print("Trial %d: meta and event stream are not aligned" % trial)
                break  
        rts = get_reaction_times(f, start_sync_t, end_sync_t)        
        data_scalar['rt'] = rts[:len(data_scalar['dur'])] ## TODO: remove indexing once you've fixed alignment prob w meta and event stream
        
        data_scalar = pd.DataFrame(data_scalar)
        return data_scalar
    
    fpath = '/Users/%s/Documents/MWorks/Data' % username
    f =  MWKFile('%s/%s' % (fpath, fn))
    f.open()
    codec = f.codec
    codenames = ['card_b', 'card_a'] ## TODO: add trial_index with new set of data    
    start_sync_t, end_sync_t = get_sync_times(f)
    print('Collecting %d trials of data' % len(start_sync_t))
    trial_dur_t = (end_sync_t - start_sync_t) / 10**6
    print('Average trial duration: %f seconds' % np.mean(trial_dur_t))
    data_scalar = get_scalar_data(f, codenames, start_sync_t, end_sync_t)
    return data_scalar
    
    

In [253]:
fn = "apiccato-concentration-20200116-143523.mwk2"
data_scalar = unpack(fn, subject_fn)    



Average trial duration: 1.583154 seconds
Trial 108: meta and event stream are not aligned
